In [62]:
import numpy as np
import pandas as pd
import math



In [44]:
import tensorflow as tf 

In [58]:
from tensorflow.keras import backend as K

In [45]:
df = pd.read_csv('C:\TPC_GAN_SIM\TPC-GAN-SIM\Data\digits.csv')

In [46]:
df.describe()

,evtId,ipad,itime,amp,crossing_angle,dip_angle,drift_length,pad_coordinate
count,544513.000000,544513.000000,544513.000000,544513.000000,544513.000000,544513.000000,544513.000000,544513.000000
mean,9998.116271,43.191593,172.717893,243.400471,-0.057631,-0.505002,172.718675,43.191624
std,5767.839085,1.540539,72.647436,639.426251,11.693803,36.664225,72.613729,1.152069
min,0.000000,39.000000,31.000000,1.000000,-20.000000,-59.985000,35.878000,41.192000
25%,5011.000000,42.000000,111.000000,7.362000,-10.357000,-33.817000,110.722000,42.207000
50%,10002.000000,43.000000,177.000000,46.470000,-0.066000,-0.654000,177.216000,43.175000
75%,14987.000000,44.000000,236.000000,260.600000,10.201000,32.684000,236.080000,44.194000
max,19999.000000,48.000000,296.000000,39990.000000,20.099000,59.993000,290.405000,45.192000


In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 544513 entries, 0 to 544512
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   evtId           544513 non-null  int64  
 1   ipad            544513 non-null  int64  
 2   itime           544513 non-null  int64  
 3   amp             544513 non-null  float64
 4   crossing_angle  544513 non-null  float64
 5   dip_angle       544513 non-null  float64
 6   drift_length    544513 non-null  float64
 7   pad_coordinate  544513 non-null  float64
dtypes: float64(5), int64(3)
memory usage: 33.2 MB


In [48]:
input_features = df.iloc[:,-4:]
input_features

,crossing_angle,dip_angle,drift_length,pad_coordinate
0,-8.695,32.201,35.936,41.844
1,-8.695,32.201,35.936,41.844
2,-8.695,32.201,35.936,41.844
3,-8.695,32.201,35.936,41.844
4,-8.695,32.201,35.936,41.844
...,...,...,...,...
544508,10.016,-12.861,52.403,42.317
544509,10.016,-12.861,52.403,42.317
544510,10.016,-12.861,52.403,42.317
544511,10.016,-12.861,52.403,42.317


In [49]:
def linearity_scaling(pd_series ,  a = -1, b = 1):
    x,y = pd_series.min() , pd_series.max()
    out = (pd_series - x) /(y-x) *(b-a) +a
    
    return out 
    

In [50]:
def preprocessing_func ( features):
    
    temp_features = pd.DataFrame()
    #it is sufficient to only feed the fractional part of the pad coordinate into our model
    temp_features['pad_coordinate'] = features['pad_coordinate'] % 1 
    
    # drift length both the fractional part and the full number are fed into the model as two separate features
    
    temp_features ['drift_length_frac'] = features['drift_length'] % 1
    
    
    
    # the angles and the drift length are linearly scaled down to a [−1, 1] region
    
    columns = ['crossing_angle','dip_angle','drift_length']
    
    for col in columns:
        temp_features[col] = linearity_scaling(features[col])
    
    
    return temp_features

In [51]:
tmp_df = preprocessing_func(input_features)
tmp_df.head()

,pad_coordinate,drift_length_frac,crossing_angle,dip_angle,drift_length
0,0.844,0.936,-0.436146,0.536715,-0.999544
1,0.844,0.936,-0.436146,0.536715,-0.999544
2,0.844,0.936,-0.436146,0.536715,-0.999544
3,0.844,0.936,-0.436146,0.536715,-0.999544
4,0.844,0.936,-0.436146,0.536715,-0.999544


In [52]:
tmp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 544513 entries, 0 to 544512
Data columns (total 5 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   pad_coordinate     544513 non-null  float64
 1   drift_length_frac  544513 non-null  float64
 2   crossing_angle     544513 non-null  float64
 3   dip_angle          544513 non-null  float64
 4   drift_length       544513 non-null  float64
dtypes: float64(5)
memory usage: 20.8 MB


In [54]:
preprocessed_input_features = tf.convert_to_tensor(tmp_df)
print (preprocessed_input_features)

tf.Tensor(
[[ 0.844       0.936      -0.43614554  0.53671506 -0.99954425]
 [ 0.844       0.936      -0.43614554  0.53671506 -0.99954425]
 [ 0.844       0.936      -0.43614554  0.53671506 -0.99954425]
 ...
 [ 0.317       0.403       0.49709469 -0.21445598 -0.8701513 ]
 [ 0.317       0.403       0.49709469 -0.21445598 -0.8701513 ]
 [ 0.317       0.403       0.49709469 -0.21445598 -0.8701513 ]], shape=(544513, 5), dtype=float64)


In [68]:

'''
custom log with base in tensorflow if there was a problem with the values
def log10(x):
  numerator = tf.log(x)
  denominator = tf.log(tf.constant(10, dtype=numerator.dtype))
  return numerator / denominator
'''

def custom_activation(x , T = math.log10(2), alpha = 0.1, gamma = 0.01 ):
    '''
    if gamma < x :
        T-gamma +x
    elif 0<x<=gamma:
        T * (alpha + ((1- alpha)*(x/gamma)))
    else :
        alpha * T * math.exp(x)
    
    '''
    return K.switch(x> gamma , T - gamma + x , K.switch(x < 0, alpha * T * tf.math.exp(x),T * (alpha + ((1- alpha)*(x/gamma)))))

In [69]:
#testing the activation function 

X = np.random.uniform(0,1, (100,10))
y = np.random.uniform(0,1, 100)

inp = tf.keras.Input((10,))
x = tf.keras.layers.Dense(8, activation=custom_activation)(inp)
out = tf.keras.layers.Dense(1)(x)

model = tf.keras.Model(inp, out)
model.compile('adam', 'mse')
model.fit(X,y, epochs=3)

Epoch 1/3
4/4 [==============================] - 0s 4ms/step - loss: 1.3717
Epoch 2/3
4/4 [==============================] - 0s 3ms/step - loss: 1.2496
Epoch 3/3
4/4 [==============================] - 0s 3ms/step - loss: 1.1297


In [84]:
#Appendix A: Generator


# attention, care that last dense layer function should be more described 
def create_generator_structure():
    
    generator = tf.keras.Sequential(
        [
            tf.keras.layers.Dense(32, activation='relu'),
            tf.keras.layers.Dense(64,activation='relu'),
            tf.keras.layers.Dense(64,activation='relu'),
            tf.keras.layers.Dense(64,activation='relu'),
            tf.keras.layers.Dense(8*16,activation=custom_activation),
            tf.keras.layers.Reshape(target_shape= (8,16))
        ],
        name= 'generator'
    )
    return generator

In [88]:
model = create_generator_structure()
model.build(input_shape= (37,1))
model.summary()

Model: "generator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_70 (Dense)            (37, 32)                  64        
                                                                 
 dense_71 (Dense)            (37, 64)                  2112      
                                                                 
 dense_72 (Dense)            (37, 64)                  4160      
                                                                 
 dense_73 (Dense)            (37, 64)                  4160      
                                                                 
 dense_74 (Dense)            (37, 128)                 8320      
                                                                 
 reshape_13 (Reshape)        (37, 8, 16)               0         
                                                                 
Total params: 18,816
Trainable params: 18,816
Non-trainab

In [ ]:
#Appendix A: discriminator

def create_discriminator_structure():
    discriminator = tf.keras.Sequential(
        [
            
        ]
    ) 